In [8]:
import xgboost as xgb
import pandas as pd
!pip install --user xgboost
import xgboost
from datetime import datetime
import numpy as np
import torch
from sklearn.preprocessing import OneHotEncoder
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [14]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

In [11]:
origin_df = pd.read_csv('metaData_taxistandsID_name_GPSlocation.csv')

In [12]:
def parse_time(x):
    dt = datetime.fromtimestamp(x["TIMESTAMP"])
    return dt.year, dt.month, dt.day, dt.hour, dt.weekday()

def polyline_to_trip_duration(polyline):
    return max(polyline.count("[") - 1, 0) * 15

import ast
def parse_stand(df):
    def extract_coordinate(polyline):
        coordinate = ast.literal_eval(polyline)[0]
        return float(coordinate[0]), float(coordinate[1])

    for index, row in df.iterrows():
        origin_stand = row['ORIGIN_STAND']
        if 'POLYLINE' not in row.index:
            if origin_stand == -100:
                continue
            else:
                origin = origin_stand
                area = origin_df.iloc[int(origin) - 1, 1]
                int1, int2 = origin_df.iloc[int(origin) - 1, 2:4]
                if int1 == '41.163066654-8.67598304213':
                    int1, int2 = '41.163066654', '-8.67598304213'
                df.at[index, 'Latitude'] = float(int1)
                df.at[index, 'Longitude'] = float(int2) 
                continue
                
        
        polyline = row['POLYLINE']
        
        if origin_stand == -100:
            if polyline:
                latitude, longitude = extract_coordinate(polyline)
                df.at[index, 'Latitude'] = latitude
                df.at[index, 'Longitude'] = longitude
        else:
            origin = origin_stand
            area = origin_df.iloc[int(origin) - 1, 1]
            int1, int2 = origin_df.iloc[int(origin) - 1, 2:4]
            if int1 == '41.163066654-8.67598304213':
                int1, int2 = '41.163066654', '-8.67598304213'
            df.at[index, 'Latitude'] = float(int1)
            df.at[index, 'Longitude'] = float(int2)

    return df

In [18]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import matplotlib.pyplot as plt

bins = [(0, 360, 0.4, 200), (360, 900, 0.4, 200), (900, 1500,0.01, 200), (1500, 2000, 0.01, 200)]

dictOne, dictTwo, dictThree, dictFour, dictFive, dictSix = {}, {}, {}, {}, {}, {}
index = 0

for idx, (lower, upper, lr, offset) in enumerate(bins):
    scaler = StandardScaler()
    
    filename = 'model' + str(idx)
    file_extension = 'model'
    file_path = f"{filename}.{file_extension}"
    model.load_model(file_path)

    # testing predictions
    modelOut = pd.read_csv('680_score.csv')
    newdf =  modelOut[(modelOut.get("TRAVEL_TIME") > lower - offset) & (modelOut.get("TRAVEL_TIME") <= upper + offset)]
    
    test_df = pd.read_csv('test_public.csv')
    mergedUnderDf = test_df.merge(newdf, left_on='TRIP_ID', right_on='TRIP_ID')
    test_df = mergedUnderDf
    
    test_df[["YR", "MON", "DAY", "HR", "WK"]] = test_df[["TIMESTAMP"]].apply(parse_time, axis=1, result_type="expand")
    
    test_df['ORIGIN_STAND'].fillna(-100.0, inplace=True)
    test_df = parse_stand(test_df)
    test_df['TAXI_ID'] = test_df['TAXI_ID'] - 20000000 
    # perform one-hot encoding
    one_hot = pd.get_dummies(test_df['CALL_TYPE'])
    # concatenate the one-hot encoded dataframe with the original dataframe
    test_df['A'] = 0
    test_df['B'] = 0
    test_df['C'] = 0
    
    if('A' in one_hot.columns):
        test_df['A'] = one_hot['A']
    if('B' in one_hot.columns):
        test_df['B'] = one_hot['B']
    if('C' in one_hot.columns):
        test_df['C'] = one_hot['C']

    tripidSeriesUnder = test_df.get('TRIP_ID')

    # Define a list of columns to be scaled
    columns_to_scale = ["TAXI_ID", "YR", "MON", "DAY", "HR", "WK", "A", "B", "C", "Latitude", "Longitude"]

    X_test = test_df.drop(["CALL_TYPE",'ORIGIN_CALL', 'DAY_TYPE', 'TRIP_ID', "TIMESTAMP", "ORIGIN_STAND", "MISSING_DATA", "TRAVEL_TIME"], axis=1)
    X_test_scaled = X_test.copy()
    # Create a scaler object and fit it on the training data
    scaler.fit(test_df[columns_to_scale])
    X_test_scaled[columns_to_scale] = scaler.transform(X_test[columns_to_scale])

    X_test_scaled_converted = xgb.DMatrix(X_test_scaled)
    y_pred_under = model.predict(X_test_scaled_converted)


    for i, val in enumerate(tripidSeriesUnder.values):
        if(index == 0):
            dictOne[val] = y_pred_under[i]
        elif(index == 1):
            dictTwo[val] = y_pred_under[i]
        elif(index == 2):
            dictThree[val] = y_pred_under[i]
        elif(index == 3):
            dictFour[val] = y_pred_under[i]
        elif(index == 4):
            dictFive[val] = y_pred_under[i]
        else:
            dictSix[val] = y_pred_under[i]
    
    index += 1

In [19]:
import re
import pandas as pd

# Combine the dictionaries and average values for duplicate keys
combined_dict = {}
all_dicts = [dictOne, dictTwo, dictThree, dictFour, dictFive, dictSix]

for dictionary in all_dicts:
    for key, value in dictionary.items():
        if key in combined_dict:
            combined_dict[key].append(value)
        else:
            combined_dict[key] = [value]

for key, values in combined_dict.items():
    average_value = sum(values) / len(values)
    combined_dict[key] = average_value

# Extract the numeric part from the keys and sort based on numeric value
sorted_keys = sorted(combined_dict.keys(), key=lambda x: int(x[1:]))

# Create a pandas Series with the sorted keys and values
s = pd.Series([combined_dict[k] for k in sorted_keys], index=sorted_keys)

print(s)

# Creating the DataFrame
data = {
    'TRIP_ID': s.index,
    'TRAVEL_TIME': s.values
}
submit_df = pd.DataFrame(data)

# Saving the DataFrame as a CSV file
submit_df.to_csv('myPreds.csv', index=False)

T1       505.035583
T2       393.028679
T3       863.954803
T4       695.635620
T5      1708.607300
           ...     
T323     603.716812
T324     870.590576
T325     406.813675
T326     404.697815
T327     593.048233
Length: 320, dtype: float64
